In [2]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns
from math import pi
import matplotlib.pyplot as plt
import random
%matplotlib inline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from xgboost import XGBClassifier

In [3]:
train_labels = pd.read_csv('Archivos de datos/train_labels.csv', encoding='latin-1')
train_values = pd.read_csv('Archivos de datos/train_values.csv', encoding='latin-1')

join = train_values.merge(train_labels)

## Experimentacion
join['damage_grade'].value_counts()


y=join['damage_grade'].astype(int)

X = join.drop(columns = 'damage_grade')

In [4]:
columnasCategoricas=np.array(['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'position',\
                                    'plan_configuration', 'legal_ownership_status','other_floor_type'  
                                   ])
columnasCategoricas

array(['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'position', 'plan_configuration',
       'legal_ownership_status', 'other_floor_type'], dtype='<U22')

In [5]:
X[columnasCategoricas] = X[columnasCategoricas].astype(dtype="category")
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [ ]:
params = {'depth':[5, 8, 10, 15],
          'iterations':[1000,5000],
          'learning_rate':[0.03,0.001,0.01,0.1], 
          'l2_leaf_reg':[3,1,5,10,50,100],
          'border_count':[32,5,10,20,50,100,200],
         
         }


clf=CatBoostClassifier(cat_features=columnasCategoricas, task_type="GPU")

kf=KFold(n_splits=3,shuffle=True)

rs=RandomizedSearchCV(clf,param_distributions=params,cv=kf,scoring='f1_micro')
rs.fit(X,y)

In [1]:
rs.best_params_

NameError: name 'rs' is not defined

In [ ]:
clf=CatBoostClassifier( learning_rate= 0.2, l2_leaf_reg=5, iterations= 1000, depth= 8, border_count= 200, cat_features=columnasCategoricas, task_type="GPU")
clf.fit(X,y)

In [14]:
X_test=pd.read_csv('Archivos de datos/test_values.csv')
X_test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


In [15]:
prediction=clf.predict(X_test)
result=pd.DataFrame(prediction)

In [16]:
result['building_id']=X_test['building_id']
result.rename(columns={0:'damage_grade'},inplace=True)
result=result[['building_id','damage_grade']]
result.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3


In [17]:
result.to_csv('Archivos de datos/resultado.csv',index=False)